In [128]:
import pandas as pd
import regex as re

In [129]:
data = pd.read_csv('log_week_end.csv')
data.rename(columns={'query': 'q'}, inplace=True)

In [130]:
data.sample(5)

,rn,loguser,q
101785,129531,dev_359,"join tbl_33570,join tbl_33595,join tbl_33623"
415481,717519,etl_2048,from tbl_142463
128502,652979,etl_2048,"from tbl_96226,into tbl_96259"
331316,921644,dev_53,from tbl_180459
494704,922155,etl_2048,"from tbl_137694,from tbl_137729"


In [133]:
data.q = data.q.str.lower().str.split(',')

Собираем в списки все команды

In [136]:
intos, froms, joins = [], [], []

In [139]:
'''
выглядит карисивее но выполняется в 42 раза медленне, чем вариант ниже
%%time
for index, row in data.iterrows():
    for x in row.q:
        if 'from' in x:
            froms.append(x)
        if 'into' in x:
            intos.append(x)
        if 'join' in x:
            joins.append(x)
'''

"\n%%time\nfor index, row in data.iterrows():\n    for x in row.q:\n        if 'from' in x:\n            froms.append(x)\n        if 'into' in x:\n            intos.append(x)\n        if 'join' in x:\n            joins.append(x)\n"

In [137]:
%%time
for x in data.q:
    for g in x:
        if 'join' in g:
            joins.append(g)
        if 'into' in g:
            intos.append(g)
            
for x in data[data.loguser.str.contains('dev')].q:
    for g in x:
        if 'from' in g:
            froms.append(g)

Wall time: 569 ms


Оставляем только название таблиц

In [142]:
intos_tbs = [re.search('tbl_\d*', x).group(0) for x in intos]
froms_tbs = [re.search('tbl_\d*', x).group(0) for x in froms]
joins_tbs = [re.search('tbl_\d*', x).group(0) for x in joins]

Делаем датафрейм с количество from (from только от пользователей dev) при помощи функции `create_df`

In [144]:
def create_df(input_data, column):
    df = pd.DataFrame(data = input_data, columns=['col']).groupby('col').agg({'col': 'count'})
    df.columns= [column]
    df.reset_index(inplace=True)
    return df

In [145]:
df1 = create_df(froms_tbs, 'count_froms')

**Делаем дф с кол-ом join**

In [147]:
df3 = create_df(joins_tbs, 'count_joins')

Объединим join и from при помощи `merge_df`:

In [148]:
def merge_df(data1, data2):
    data = data1.merge(data2, on ='col', how='outer')
    data = data.fillna(0)
    return data

In [149]:
df1 = merge_df(df1,df3)
df1.head()

,col,count_froms,count_joins
0,tbl_100371,4.0,0.0
1,tbl_100498,2.0,0.0
2,tbl_100584,2.0,11.0
3,tbl_100776,1.0,0.0
4,tbl_10080,8.0,13.0


**Делаем дф с кол-ом into**

In [161]:
df2.count_intos = df2.count_intos + 1

In [176]:
df2 = create_df(intos_tbs, 'count_intos')
df = merge_df(df1,df2)
df.count_intos = df.count_intos + 1
df['usefull'] = df['count_intos'] / df['count_froms']
df.fillna(0, inplace=True)
df.head()

,col,count_froms,count_joins,count_intos,usefull
0,tbl_100371,4.0,0.0,1.0,0.250
1,tbl_100498,2.0,0.0,1.0,0.500
2,tbl_100584,2.0,11.0,11.0,5.500
3,tbl_100776,1.0,0.0,1.0,1.000
4,tbl_10080,8.0,13.0,1.0,0.125


In [179]:
pd.set_option('mode.use_inf_as_na', True) #включаем опцию, где inf (бесконечность) считаем как NaN
#предлагаю удалить

In [178]:
df.sort_values(by = 'usefull', ascending = False, inplace = True)
df.head(10)

,col,count_froms,count_joins,count_intos,usefull
1463,tbl_211513,8.0,1.0,9594.0,1199.250000
1692,tbl_29455,1.0,1595.0,471.0,471.000000
1636,tbl_26776,1.0,3083.0,469.0,469.000000
1644,tbl_27182,1.0,8208.0,465.0,465.000000
1462,tbl_211512,21.0,0.0,3139.0,149.476190
652,tbl_15647,2.0,0.0,203.0,101.500000
1919,tbl_44933,10.0,0.0,881.0,88.100000
1520,tbl_23594,1.0,0.0,54.0,54.000000
1466,tbl_211522,32.0,0.0,1171.0,36.593750
1548,tbl_23770,12.0,0.0,325.0,27.083333
